In [1]:
import requests
import json
import os



In [2]:

def get_next_page(response):
    paging = response.get("paging", {})
    next_url = paging.get("next", None)
    return next_url


In [3]:

def main(api_url):
    all_data = []

    while api_url:
        response = requests.get(api_url)
        if response.status_code == 200:
            json_data = response.json()
            data = json_data.get("data", [])
            all_data.extend(data)
            api_url = get_next_page(json_data)
        else:
            print("Error:", response.status_code)
            break

    return all_data

In [6]:
output_dir = "political_ads_delta"  # Directory to save the JSON files

bylines = ['ANO']

for byline in bylines:
    # fields = "id,ad_snapshot_url,ad_creation_time,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,ad_creative_link_titles,ad_delivery_start_time,ad_delivery_stop_time,bylines,currency,delivery_by_region,demographic_distribution,estimated_audience_size,impressions,languages,page_id,page_name,publisher_platforms,spend,target_locations,target_gender,target_ages,eu_total_reach,beneficiary_payers,age_country_gender_reach_breakdown"
    # supported_countries = ["CZ"]
    access_token = "EAALnc8im5MUBOZBXcyevhADuqZBvZBT8VZBr3bmodjXeycutV6XsFmxWa7BFKeK5UpFJ3RDgybaaweneTgXn8B0j94fxRHoUpm0VmHv9cMawbupl7bCERO7mEO43ftZBIMKxuZALmb4wCZBd7iZBxz7C8mYhE0HSOf148ZAH3rZAusnz9hCGfwn0EI2hZBJdolB4SFn0wVuyWTN"
    api_url = f"https://graph.facebook.com/v21.0/ads_archive?bylines={bylines}&ad_type=POLITICAL_AND_ISSUE_ADS&ad_reached_countries=['CZ']&access_token={access_token}&unmask_removed_content=true&fields=id,ad_snapshot_url,ad_creation_time,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,ad_creative_link_titles,ad_delivery_start_time,ad_delivery_stop_time,bylines,currency,delivery_by_region,demographic_distribution,estimated_audience_size,impressions,languages,page_id,page_name,publisher_platforms,spend,target_locations,target_gender,target_ages,eu_total_reach,beneficiary_payers,age_country_gender_reach_breakdown&limit=999&search_terms=''&languages=['cs']"
    extracted_data = main(api_url)

    # Write the extracted data to a JSON file
    filename = os.path.join(output_dir, f"data_{byline}.json")
    with open(filename, "w", encoding='utf-8') as json_file:
                json.dump(extracted_data, json_file, indent=4, ensure_ascii=False)

    print(f"All Extracted Data written to {filename}")


All Extracted Data written to political_ads_delta\data_ANO.json


In [6]:
import os
import json

def replace_id_with_underscore_id(json_data):
    if isinstance(json_data, dict):
        for key, value in list(json_data.items()):
            if key == "id":
                json_data["_id"] = json_data.pop("id")
            replace_id_with_underscore_id(value)
    elif isinstance(json_data, list):
        for item in json_data:
            replace_id_with_underscore_id(item)

def replace_id_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            
            with open(file_path, "r", encoding="utf-8") as json_file:
                data = json.load(json_file)

            replace_id_with_underscore_id(data)

            with open(file_path, "w", encoding="utf-8") as json_file:
                json.dump(data, json_file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    folder_path = "political_ads_delta"  # Replace with the actual path to your folder
    replace_id_in_folder(folder_path)


c:\Users\Jiri.Pillar\Developer\Python\meta_ads_crawler


### push data to mongo

update if exists

insert if not


In [7]:
from pymongo import MongoClient

# Connection to MongoDB
client = MongoClient('mongodb://localhost:27017')

# Select your database
db = client['meta_ads_db']

# Select your collection
collection = db['meta_ads_collection']

# Specify the folder containing JSON files
folder_path = 'political_ads_delta'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        # Load data from JSON file with 'utf-8' encoding
        with open(file_path, encoding='utf-8') as file:
            data = json.load(file)

        # Upsert each document from the JSON file
        for document in data:
            result = collection.update_one(
                {"_id": document["_id"]},
                {"$set": document},
                upsert=True
            )
        # Print the total number of documents added or updated for the current file
        print(f"File: {filename}, Total documents processed: {len(data)}")


# Close the connection
client.close()

File: data_100136888779860.json, Total documents processed: 2
File: data_100174253112291.json, Total documents processed: 4
File: data_100201752249169.json, Total documents processed: 8
File: data_100202116270600.json, Total documents processed: 0
File: data_100247113133632.json, Total documents processed: 0
File: data_100423156141323.json, Total documents processed: 0
File: data_100424363151861.json, Total documents processed: 0
File: data_100505211433010.json, Total documents processed: 3
File: data_100537783337910.json, Total documents processed: 0
File: data_100602322206023.json, Total documents processed: 6
File: data_100611015570191.json, Total documents processed: 1
File: data_100617488241201.json, Total documents processed: 8
File: data_1006411482778121.json, Total documents processed: 0
File: data_100647378751279.json, Total documents processed: 2
File: data_100662591864125.json, Total documents processed: 0
File: data_100683176313.json, Total documents processed: 0
File: data

## drop entire db

In [ ]:
# from pymongo import MongoClient
# 
# # Connection to MongoDB
# client = MongoClient('mongodb://localhost:27017')
# 
# # Select your database
# db = client['meta_ads_db']
# 
# # Select your collection
# collection = db['meta_ads_collection']
# 
# # Truncate (remove all documents from) the collection
# result = collection.delete_many({})
# 
# # Print the number of deleted documents
# print(f"Number of documents deleted: {result.deleted_count}")
# 
# # Close the connection
# client.close()
